In [1]:
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB # modele d'exemple a modifier
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import pickle

In [3]:
# Import data
data = pd.read_csv('data/train_diabetes_health_indicators.csv')
data['Diabetes_012'] = data['Diabetes_012'].astype(int)
data = data.drop(columns=['Unnamed: 0'])
X = data.drop(columns=['Diabetes_012'])
y = data['Diabetes_012']
